In [1]:
!git clone -b physics-informed --single-branch \
https://github.com/oskareriksandelin/p30gnn.git


Cloning into 'p30gnn'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 107 (delta 21), reused 99 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 62.21 KiB | 4.44 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00a 0:00:01


In [3]:
!pwd
!ls


/kaggle/working
p30gnn


## LOAD THE DATA

In [4]:
!pip -q install gdown   

In [5]:
import gdown

In [6]:
folder_url = "https://drive.google.com/drive/folders/1y10KJqDBL5RhETXMChe48e93Q0oubGHn?usp=drive_link"
gdown.download_folder(folder_url, output="data", quiet=False, remaining_ok=True)

Retrieving folder contents


Processing file 19HVQoNILBqnfv7vvSdcTXirFptHNVlVX FeGd_data_POSCAR_2.zip
Processing file 15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT FeGd_data_POSCAR_3.zip
Processing file 1f__wyGmIioGC9WfimVaypb6HrmuJ06vq FeGd_data_POSCAR_4.zip
Processing file 1OmYIMRib6yYpWmxcagvhKVUVHJfoiVwh FeGd_data_POSCAR_5.zip
Processing file 1x71MDnwVG6QQ4Nm9Gan6Y2uUo3TDoIFG FeGd_data_POSCAR_6.zip
Processing file 1F4OvZB1U2P1OEOxvGdr_-6CnbpVpjrRi FeGd_data_POSCAR_7.zip
Processing file 1ybzjthY_8ZHUOzcjQ62Du9v24EW4Azct FeGd_data_POSCAR_8.zip
Processing file 1baE5wY5rFIQERD0l2tJFBK2vCtyQ2c5x FeGd_data_POSCAR_9.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=19HVQoNILBqnfv7vvSdcTXirFptHNVlVX
From (redirected): https://drive.google.com/uc?id=19HVQoNILBqnfv7vvSdcTXirFptHNVlVX&confirm=t&uuid=55042f53-9183-4093-81f2-d4c8714dae56
To: /kaggle/working/data/FeGd_data_POSCAR_2.zip
100%|██████████| 278M/278M [00:02<00:00, 114MB/s]  
Downloading...
From (original): https://drive.google.com/uc?id=15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT
From (redirected): https://drive.google.com/uc?id=15lcB99k6cYbt0HGnQj_g8QXtb3N2nCFT&confirm=t&uuid=60367e4b-7fbd-4cff-a7fb-8ab54fd53588
To: /kaggle/working/data/FeGd_data_POSCAR_3.zip
100%|██████████| 34.7M/34.7M [00:00<00:00, 62.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1f__wyGmIioGC9WfimVaypb6HrmuJ06vq
From (redirected): https://drive.google.com/uc?id=1f__wyGmIioGC9WfimVaypb6HrmuJ06vq&confirm=t&uuid=9264d247-4a08-4f76-97b4-83ecf44d

['data/FeGd_data_POSCAR_2.zip',
 'data/FeGd_data_POSCAR_3.zip',
 'data/FeGd_data_POSCAR_4.zip',
 'data/FeGd_data_POSCAR_5.zip',
 'data/FeGd_data_POSCAR_6.zip',
 'data/FeGd_data_POSCAR_7.zip',
 'data/FeGd_data_POSCAR_8.zip',
 'data/FeGd_data_POSCAR_9.zip']

In [7]:
!pwd
!ls

/kaggle/working
data  p30gnn


In [8]:
!ls data

FeGd_data_POSCAR_2.zip	FeGd_data_POSCAR_5.zip	FeGd_data_POSCAR_8.zip
FeGd_data_POSCAR_3.zip	FeGd_data_POSCAR_6.zip	FeGd_data_POSCAR_9.zip
FeGd_data_POSCAR_4.zip	FeGd_data_POSCAR_7.zip


In [9]:
import zipfile
from pathlib import Path

data_path = Path("/kaggle/working/data")

for z in data_path.glob("*.zip"):
    with zipfile.ZipFile(z) as f:
        f.extractall(data_path)
    z.unlink()  # remove the zip file

In [10]:
!ls data

FeGd_data_POSCAR_2  FeGd_data_POSCAR_4	FeGd_data_POSCAR_6  FeGd_data_POSCAR_8
FeGd_data_POSCAR_3  FeGd_data_POSCAR_5	FeGd_data_POSCAR_7  FeGd_data_POSCAR_9


## Libraries

In [11]:
from p30gnn.FeGdSpinGNN.utils import normalize_data, compute_normalization_stats
from p30gnn.FeGdSpinGNN.FeGd_dataset import FeGdMagneticDataset
from p30gnn.FeGdSpinGNN.models import NaiveFeGdBFieldModel


In [12]:
def normalize_dataset(dataset, stats):
    normalized_list = []
    for data in dataset:
        normalized_list.append(normalize_data(data, stats))
    return normalized_list


train_dataset = FeGdMagneticDataset(data_path, systems=[2, 3, 4, 5], cutoff_dist=0.51)
val_dataset = FeGdMagneticDataset(data_path, systems=[6, 7], cutoff_dist=0.51)
test_dataset = FeGdMagneticDataset(data_path, systems=[8, 9], cutoff_dist=0.51)

# compute normalization stats
norm_stats = compute_normalization_stats(train_dataset)

train_data_norm = normalize_dataset(train_dataset, norm_stats)
val_data_norm = normalize_dataset(val_dataset, norm_stats)
test_data_norm = normalize_dataset(test_dataset, norm_stats)

Loading systems:   0%|          | 0/4 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/data/fields/POSCAR_2/bintefftot.FeGd_100.out'